<a href="https://colab.research.google.com/github/dhoumaima/SugarSense/blob/master/SugarSenseML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#installer pyspark

In [ ]:
!pip install pyspark

#Initialisation d'une session Spark pour la prédiction du diabète

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DiabetesPrediction").getOrCreate()


#Chargement et exploration des données de diabète avec PySpark

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
file_path = "/content/drive/MyDrive/diabetes.csv"

# Charger les données dans un DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.groupBy("Outcome").count().show()


+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  268|
|      0|  500|
+-------+-----+



#Extraction et sauvegarde d'un échantillon de 20% des données avec PySpark
pour la partie de validation

In [4]:
from pyspark.sql import SparkSession

# Créer une session Spark
spark = SparkSession.builder.appName("Save20Percent").getOrCreate()

# Charger le fichier CSV
data = spark.read.csv("/content/drive/MyDrive/diabetes.csv", header=True, inferSchema=True)

# Diviser les données pour prendre 20%
df1, sample_data = data.randomSplit([0.8, 0.2], seed=1234)

# Sauvegarder ces 20% dans un fichier CSV dans Google Drive
sample_data.write.csv("/content/drive/MyDrive/diabetes_sample_20.csv", header=True, mode="overwrite")
df1.write.csv("/content/drive/MyDrive/diabetes_sample_80.csv", header=True, mode="overwrite")
# Afficher les premières lignes pour vérifier
sample_data.show(5)


+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          0|     67|           76|            0|      0|45.3|                   0.194| 46|      0|
|          0|     78|           88|           29|     40|36.9|                   0.434| 21|      0|
|          0|    100|           70|           26|     50|30.8|                   0.597| 21|      0|
|          0|    101|           64|           17|      0|21.0|                   0.252| 21|      0|
|          0|    101|           65|           28|      0|24.6|                   0.237| 22|      0|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows



#Classification du diabète à l'aide d'un arbre de décision avec PySpark

In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

# Créer une session Spark
spark = SparkSession.builder.appName("DecisionTreeDiabetes").getOrCreate()

# Charger le fichier CSV
data = spark.read.csv("/content/drive/MyDrive/diabetes_sample_80.csv", header=True, inferSchema=True)

# Afficher les premières lignes du DataFrame pour vérifier les données
data.show(5)

# Liste des colonnes à utiliser comme caractéristiques
feature_columns = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI",
                   "DiabetesPedigreeFunction", "Age"]

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Définir l'arbre de décision
dt = DecisionTreeClassifier(labelCol="Outcome", featuresCol="features", predictionCol="prediction")

# Construire le pipeline
pipeline = Pipeline(stages=[assembler, dt])

# Diviser les données en ensemble d'entraînement et de test
train_data, test_data = data.randomSplit([0.8, 0.2], seed=1234)

# Entraîner le modèle
model = pipeline.fit(train_data)

# Appliquer les prédictions sur les données de test
predictions = model.transform(test_data)

# Afficher les résultats de prédiction
predictions.select("prediction", "Outcome", "features").show(5)

# Évaluation du modèle avec AUC
evaluator = BinaryClassificationEvaluator(labelCol="Outcome", rawPredictionCol="prediction")

# Calculer l'AUC (Area Under Curve)
auc = evaluator.evaluate(predictions)
print(f"AUC: {auc}")


model.write().overwrite().save("decision_tree_model_diabetes")


+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          0|     57|           60|            0|      0|21.7|                   0.735| 67|      0|
|          0|     73|            0|            0|      0|21.1|                   0.342| 25|      0|
|          0|     74|           52|           10|     36|27.8|                   0.269| 22|      0|
|          0|     84|           64|           22|     66|35.8|                   0.545| 21|      0|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows

+----------+-------+--------------------+
|prediction|Outcome|            f

#Séparation des caractéristiques et des résultats à partir d'un jeu de données de diabète avec PySpark

In [6]:
from pyspark.sql import SparkSession

# Créer une session Spark
spark = SparkSession.builder.appName("SplitOutcome").getOrCreate()

# Charger le fichier CSV contenant l'échantillon de données
data = spark.read.csv("/content/drive/MyDrive/diabetes_sample_20.csv", header=True, inferSchema=True)

# Sélectionner uniquement la colonne Outcome
outcome_data = data.select("Outcome")

# Sélectionner toutes les autres colonnes sauf Outcome
features_data = data.drop("Outcome")

# Sauvegarder la colonne Outcome dans un fichier CSV
outcome_data.write.csv("/content/drive/MyDrive/outcome_data.csv", header=True, mode="overwrite")

# Sauvegarder les autres entrées (features) dans un autre fichier CSV
features_data.write.csv("/content/drive/MyDrive/features_data.csv", header=True, mode="overwrite")

# Afficher les premières lignes pour vérifier
outcome_data.show(5)
features_data.show(5)


+-------+
|Outcome|
+-------+
|      0|
|      0|
|      0|
|      0|
|      0|
+-------+
only showing top 5 rows

+-----------+-------+-------------+-------------+-------+----+------------------------+---+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+
|          0|     67|           76|            0|      0|45.3|                   0.194| 46|
|          0|     78|           88|           29|     40|36.9|                   0.434| 21|
|          0|    100|           70|           26|     50|30.8|                   0.597| 21|
|          0|    101|           64|           17|      0|21.0|                   0.252| 21|
|          0|    101|           65|           28|      0|24.6|                   0.237| 22|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+
only showing top 5 rows



#Application d'un modèle de forêt aléatoire pour la prédiction sur des données de diabète avec PySpark
parti de validation

In [7]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row

# Créer une session Spark
spark = SparkSession.builder.appName("ModelTesting").getOrCreate()

# Charger les données d'entrée (features) depuis le fichier CSV
features_data = spark.read.csv("/content/drive/MyDrive/features_data.csv", header=True, inferSchema=True)

# Vérifier le schéma des données d'entrée pour s'assurer que tout est correct
features_data.printSchema()

# Appliquer le modèle directement aux données
assembler = VectorAssembler(
    inputCols=["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin",
               "BMI", "DiabetesPedigreeFunction", "Age"],
    outputCol="feature"
)

# Appliquer le VectorAssembler pour transformer les données en un vecteur de caractéristiques
assembled_data = assembler.transform(features_data)

# Appliquer le modèle pour obtenir les prédictions
predictions = model.transform(assembled_data)

predictions_result = predictions.select("prediction")

# Afficher les premières lignes des prédictions
predictions_result.show()

# Sauvegarder les résultats dans un fichier CSV
predictions_result.write.csv("/content/drive/MyDrive/predictions_data.csv", header=True, mode="overwrite")


root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)

+----------+
|prediction|
+----------+
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       1.0|
|       0.0|
|       0.0|
|       1.0|
|       0.0|
|       0.0|
|       0.0|
+----------+
only showing top 20 rows



# Évaluation des performances d'un modèle de classification avec PySpark

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id

# Initialiser Spark
spark = SparkSession.builder.appName("MetricsCalculation").getOrCreate()

# Charger les données
predictions_df = spark.read.csv("/content/drive/MyDrive/predictions_data.csv", header=True, inferSchema=True)
outcome_df = spark.read.csv("/content/drive/MyDrive/outcome_data.csv", header=True, inferSchema=True)

# Ajouter un identifiant unique pour aligner les deux DataFrames
predictions_df = predictions_df.withColumn("id", monotonically_increasing_id())
outcome_df = outcome_df.withColumn("id", monotonically_increasing_id())

# Combiner les deux DataFrames
comparison_df = predictions_df.join(outcome_df, on="id", how="inner").drop("id")
comparison_df.show()
from pyspark.sql.functions import col

# Ajouter une colonne pour comparer les prédictions et les cibles
comparison_df = comparison_df.withColumn(
    "is_correct", (col("prediction") == col("Outcome")).cast("integer")
)

# Taux d'erreur
error_rate = 1 - (comparison_df.groupBy().sum("is_correct").first()[0] / comparison_df.count())
print(f"Taux d'erreur: {error_rate:.2f}")
from pyspark.sql.functions import when, col

# Calcul des métriques
comparison_df = comparison_df.withColumn(
    "TP", when((col("prediction") == 1) & (col("Outcome") == 1), 1).otherwise(0)
).withColumn(
    "FP", when((col("prediction") == 1) & (col("Outcome") == 0), 1).otherwise(0)
).withColumn(
    "FN", when((col("prediction") == 0) & (col("Outcome") == 1), 1).otherwise(0)
).withColumn(
    "TN", when((col("prediction") == 0) & (col("Outcome") == 0), 1).otherwise(0)
)

# Somme des métriques
metrics = comparison_df.groupBy().sum("TP", "FP", "FN", "TN").first()
TP, FP, FN, TN = metrics["sum(TP)"], metrics["sum(FP)"], metrics["sum(FN)"], metrics["sum(TN)"]

# Précision, rappel et F1-score
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Précision: {precision:.2f}")
print(f"Rappel: {recall:.2f}")
print(f"F1-Score: {f1_score:.2f}")



+----------+-------+
|prediction|Outcome|
+----------+-------+
|       0.0|      0|
|       0.0|      0|
|       0.0|      0|
|       0.0|      0|
|       0.0|      0|
|       0.0|      0|
|       0.0|      0|
|       0.0|      0|
|       0.0|      0|
|       0.0|      0|
|       0.0|      0|
|       0.0|      0|
|       0.0|      0|
|       1.0|      1|
|       0.0|      0|
|       0.0|      0|
|       1.0|      1|
|       0.0|      1|
|       0.0|      1|
|       0.0|      0|
+----------+-------+
only showing top 20 rows

Taux d'erreur: 0.28
Précision: 0.63
Rappel: 0.62
F1-Score: 0.62


#chatboot
Un chatboot pour utiliser le modéle avec des nouvaux données


In [18]:
# Importer les bibliothèques nécessaires
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel

# Initialisation de Spark
spark = SparkSession.builder.appName("DiabetesChatbot").getOrCreate()

# Charger le modèle sauvegardé
model = PipelineModel.load("decision_tree_model_diabetes")

# Fonction de prédiction
def predict_diabetes(input_data):
    # Préparer les données pour Spark
    data = {
        "Pregnancies": int(input_data[0]),
        "Glucose": float(input_data[1]),
        "BloodPressure": float(input_data[2]),
        "SkinThickness": float(input_data[3]),
        "Insulin": float(input_data[4]),
        "BMI": float(input_data[5]),
        "DiabetesPedigreeFunction": float(input_data[6]),
        "Age": int(input_data[7])
    }
    # Créer un DataFrame Spark
    spark_df = spark.createDataFrame([data])

    # Appliquer le modèle pour effectuer la prédiction
    predictions = model.transform(spark_df)
    result = predictions.collect()[0]['prediction']

    return "Diabétique" if result == 1 else "Non diabétique"

# Chatbot interactif
def chatbot():
    print("Bonjour ! Bienvenue sur le chatbot de prédiction du diabète.")
    print("Veuillez entrer les données suivantes séparées par des virgules :")
    print("Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age")

    while True:
        user_input = input("Vos données (ou tapez 'quit' pour quitter) : ")

        if user_input.lower() == 'quit':
            print("Merci d'avoir utilisé le chatbot. À bientôt !")
            break

        try:
            # Transformer l'entrée utilisateur en liste
            input_data = user_input.split(",")
            if len(input_data) != 8:
                print("Erreur : Veuillez entrer exactement 8 valeurs.")
                continue

            # Prédiction
            result = predict_diabetes(input_data)
            print(f"Résultat : {result}")
        except Exception as e:
            print(f"Une erreur est survenue : {e}")

# Exécuter le chatbot
if __name__ == "__main__":
    chatbot()


Bonjour ! Bienvenue sur le chatbot de prédiction du diabète.
Veuillez entrer les données suivantes séparées par des virgules :
Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age
Vos données (ou tapez 'quit' pour quitter) : 0,67,76,0,0,45.3,0.194,46
Résultat : Non diabétique
Vos données (ou tapez 'quit' pour quitter) : 1,50,80,0,1,52,0.122,25
Résultat : Diabétique
Vos données (ou tapez 'quit' pour quitter) : quit
Merci d'avoir utilisé le chatbot. À bientôt !


#télécharger le modèle  avec PySpark sur votre machine locale

In [ ]:
!ls -l decision_tree_model_diabetes


total 8
drwxr-xr-x 2 root root 4096 Dec  1 10:01 metadata
drwxr-xr-x 4 root root 4096 Dec  1 10:01 stages


In [ ]:
!zip -r decision_tree_model_diabetes.zip decision_tree_model_diabetes


  adding: decision_tree_model_diabetes/ (stored 0%)
  adding: decision_tree_model_diabetes/metadata/ (stored 0%)
  adding: decision_tree_model_diabetes/metadata/.part-00000.crc (stored 0%)
  adding: decision_tree_model_diabetes/metadata/part-00000 (deflated 23%)
  adding: decision_tree_model_diabetes/metadata/._SUCCESS.crc (stored 0%)
  adding: decision_tree_model_diabetes/metadata/_SUCCESS (stored 0%)
  adding: decision_tree_model_diabetes/stages/ (stored 0%)
  adding: decision_tree_model_diabetes/stages/0_VectorAssembler_b39a9b85c7d5/ (stored 0%)
  adding: decision_tree_model_diabetes/stages/0_VectorAssembler_b39a9b85c7d5/metadata/ (stored 0%)
  adding: decision_tree_model_diabetes/stages/0_VectorAssembler_b39a9b85c7d5/metadata/.part-00000.crc (stored 0%)
  adding: decision_tree_model_diabetes/stages/0_VectorAssembler_b39a9b85c7d5/metadata/part-00000 (deflated 35%)
  adding: decision_tree_model_diabetes/stages/0_VectorAssembler_b39a9b85c7d5/metadata/._SUCCESS.crc (stored 0%)
  adding

In [ ]:
!ls -l decision_tree_model_diabetes.zip


-rw-r--r-- 1 root root 10262 Dec  1 10:10 decision_tree_model_diabetes.zip


In [ ]:
from google.colab import files

files.download("decision_tree_model_diabetes.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>